In [15]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE
from skopt import BayesSearchCV
from skopt.space import Real, Categorical
from sklearn.metrics import accuracy_score

# Load dataset (Replace 'your_data.csv' with the actual filename)
df = pd.read_csv('I:/adhd/datasets/combined_featuresECEO.csv')

# Separate features and target
X = df.drop(columns=['ADHD'])  # Assuming 'target' is the label column
y = df['ADHD']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y, shuffle=True)

# Apply SMOTE to balance the dataset
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Apply Feature Scaling (Standardization)
scaler = StandardScaler()
X_train_smote_scaled = scaler.fit_transform(X_train_smote)
X_test_scaled = scaler.transform(X_test)

# Define Logistic Regression model
log_reg = LogisticRegression(max_iter=5000)

# Define hyperparameter search space for Bayesian Optimization
param_grid = {
    'C': Real(1e-4, 1e+4, prior='log-uniform'),  # Regularization strength
    'penalty': Categorical(['l1', 'l2']),  # Lasso (L1) and Ridge (L2)
    'solver': Categorical(['liblinear', 'saga'])  # Compatible solvers
}

# Perform hyperparameter tuning using Bayesian Optimization
bayes_search = BayesSearchCV(log_reg, param_grid, n_iter=50, cv=5, scoring='accuracy', n_jobs=-1, random_state=42)
bayes_search.fit(X_train_smote_scaled, y_train_smote)

# Best model from the search
best_model = bayes_search.best_estimator_

# Evaluate on test set
y_pred = best_model.predict(X_test_scaled)
test_accuracy = accuracy_score(y_test, y_pred)

print("Best Parameters:", bayes_search.best_params_)
print("Test Accuracy:", test_accuracy)


i:\Python313\Lib\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point [10000.0, np.str_('l2'), np.str_('liblinear')] before, using random point [1608.9591813418983, 'l1', 'saga']
  warnings.warn(
i:\Python313\Lib\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point [10000.0, np.str_('l2'), np.str_('liblinear')] before, using random point [0.02165300798216549, 'l1', 'saga']
  warnings.warn(
i:\Python313\Lib\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point [10000.0, np.str_('l2'), np.str_('liblinear')] before, using random point [0.0007668009901748293, 'l2', 'liblinear']
  warnings.warn(
i:\Python313\Lib\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point [10000.0, np.str_('l2'), np.str_('liblinear')] before, using random point [10.01167150555204, 'l2', 'saga']
  warnings.warn(
i:\Pytho

Best Parameters: OrderedDict({'C': 3650.009666381097, 'penalty': 'l2', 'solver': 'liblinear'})
Test Accuracy: 0.5102040816326531


Logistic Regression Hyperparameter Tuning

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.utils import resample
from sklearn.metrics import accuracy_score, classification_report
from scipy.stats import loguniform


# Load dataset (replace with your actual dataframe)
df = pd.read_csv("I:/adhd/datasets/combined_featuresECEO.csv")  # Ensure labels are in 'label' column

# Separate features and target
X = df.drop(columns=['ADHD'])  # Feature columns
y = df['ADHD']  # Target column

# Count the samples in each class
class_counts = y.value_counts()
minority_class = class_counts.idxmin()
majority_class = class_counts.idxmax()

# Separate majority and minority classes
df_majority = df[df['ADHD'] == majority_class]
df_minority = df[df['ADHD'] == minority_class]

# Undersample majority class
df_majority_undersampled = resample(df_majority, 
                                    replace=False,  # No replacement
                                    n_samples=len(df_minority),  # Match minority count
                                    random_state=42)  # Reproducibility

# Combine undersampled majority and minority class
df_balanced = pd.concat([df_minority, df_majority_undersampled])

# Shuffle dataset
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

# Split dataset
X_balanced = df_balanced.drop(columns=['ADHD'])
y_balanced = df_balanced['ADHD']
X_train, X_test, y_train, y_test = train_test_split(X_balanced, y_balanced, test_size=0.2, random_state=42)

# Define model
log_reg = LogisticRegression(solver='saga', max_iter=1000)

# Define hyperparameter grid
param_grid = {
    'C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100],  # Log-scale regularization strength
    'penalty': ['l1', 'l2', 'elasticnet'],  # Regularization type
    'l1_ratio': np.linspace(0, 1, 10),  # Only used for elasticnet
    'class_weight': [None, 'balanced'],  # Handling class imbalance
    'fit_intercept': [True, False],  # Whether to fit an intercept
    'tol': [1e-4, 1e-3, 1e-2]  # Convergence tolerance
}

# Perform Grid Search
grid_search = GridSearchCV(log_reg, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Best hyperparameters and accuracy
best_model = grid_search.best_estimator_
print("Best Hyperparameters:", grid_search.best_params_)

# Evaluate on test set
y_pred = best_model.predict(X_test)
print("Test Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


Best Hyperparameters: {'C': 10, 'class_weight': 'balanced', 'fit_intercept': False, 'l1_ratio': np.float64(0.2222222222222222), 'penalty': 'elasticnet', 'tol': 0.01}
Test Accuracy: 0.42105263157894735
Classification Report:
               precision    recall  f1-score   support

           0       0.40      0.44      0.42        18
           1       0.44      0.40      0.42        20

    accuracy                           0.42        38
   macro avg       0.42      0.42      0.42        38
weighted avg       0.42      0.42      0.42        38



In [9]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
from skopt import BayesSearchCV
from skopt.space import Real, Categorical

# Load dataset (replace with your actual dataframe)
df = pd.read_csv("I:/adhd/datasets/combined_featuresECEO.csv")  # Ensure 'label' column contains target values

# Separate features and target
X = df.drop(columns=['ADHD'])  # Feature columns
y = df['ADHD']  # Target column

# Handle class imbalance using undersampling
class_counts = y.value_counts()
minority_class = class_counts.idxmin()
majority_class = class_counts.idxmax()

df_majority = df[df['ADHD'] == majority_class]
df_minority = df[df['ADHD'] == minority_class]

df_majority_undersampled = resample(df_majority, 
                                    replace=False, 
                                    n_samples=len(df_minority), 
                                    random_state=42)  

df_balanced = pd.concat([df_minority, df_majority_undersampled])
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

# Split dataset
X_balanced = df_balanced.drop(columns=['ADHD'])
y_balanced = df_balanced['ADHD']
X_train, X_test, y_train, y_test = train_test_split(X_balanced, y_balanced, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define SVM model
svm = SVC(probability=True, random_state=42)

# Define search space for Bayesian Optimization
search_space = {
    'C': Real(1e-3, 100, prior='log-uniform'),  # Regularization parameter
    'gamma': Real(1e-4, 10, prior='log-uniform'),  # Kernel coefficient
    'kernel': Categorical(['linear', 'rbf', 'poly', 'sigmoid']),  # Kernel type
    'class_weight': Categorical([None, 'balanced'])  # Handling class imbalance
}

# Perform Bayesian Optimization
bayes_search = BayesSearchCV(
    svm, search_space, 
    n_iter=50,  # Number of parameter evaluations
    cv=5, 
    scoring='accuracy', 
    n_jobs=-1, 
    random_state=42
)
bayes_search.fit(X_train, y_train)

# Get the best model
best_model = bayes_search.best_estimator_
print("Best Hyperparameters:", bayes_search.best_params_)

# Evaluate on test set
y_pred = best_model.predict(X_test)
y_proba = best_model.predict_proba(X_test)[:, 1]

print("Test Accuracy:", accuracy_score(y_test, y_pred))
print("ROC AUC Score:", roc_auc_score(y_test, y_proba))
print("Classification Report:\n", classification_report(y_test, y_pred))


Best Hyperparameters: OrderedDict({'C': 0.18836566424305928, 'class_weight': 'balanced', 'gamma': 0.07336496078868379, 'kernel': 'sigmoid'})
Test Accuracy: 0.6052631578947368
ROC AUC Score: 0.6694444444444445
Classification Report:
               precision    recall  f1-score   support

           0       0.57      0.67      0.62        18
           1       0.65      0.55      0.59        20

    accuracy                           0.61        38
   macro avg       0.61      0.61      0.60        38
weighted avg       0.61      0.61      0.60        38



In [ ]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
from imblearn.over_sampling import SMOTE
from skopt import BayesSearchCV
from skopt.space import Real, Categorical
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
import seaborn as sns


def plot_confusion_matrix(y_true, y_pred, class_names):
    cm = confusion_matrix(y_true, y_pred)
    
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
    
    plt.xlabel("Predicted Labels")
    plt.ylabel("True Labels")
    plt.title("Confusion Matrix")
    plt.show()




def plot_roc_curve(y_true, y_probs):
    fpr, tpr, _ = roc_curve(y_true, y_probs)
    roc_auc = auc(fpr, tpr)

    plt.figure(figsize=(6, 5))
    plt.plot(fpr, tpr, color='blue', label=f'AUC = {roc_auc:.2f}')
    plt.plot([0, 1], [0, 1], color="grey", linestyle="--")  # Diagonal line

    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("ROC Curve")
    plt.legend()
    plt.show()

# Load dataset (replace with your actual dataframe)
df = pd.read_csv("I:/adhd/datasets/combined_featuresECEO.csv")  # Ensure 'label' column contains target values

# Separate features and target
X = df.drop(columns=['ADHD'])  # Feature columns
y = df['ADHD']  # Target column

# Split dataset before applying SMOTE (to prevent data leakage)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y, shuffle=True)


# Apply SMOTE to training data only
smote = SMOTE(sampling_strategy='auto', random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

scaler = StandardScaler()
X_train_smote_scaled= scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define SVM model
svm = SVC(probability=True, random_state=42)

# Define search space for Bayesian Optimization
search_space = {
    'C': Real(1e-3, 100, prior='log-uniform'),  # Regularization parameter
    'gamma': Real(1e-4, 10, prior='log-uniform'),  # Kernel coefficient
    'kernel': Categorical(['linear', 'rbf', 'poly', 'sigmoid']),  # Kernel type
    'class_weight': Categorical([None, 'balanced'])  # Handling class imbalance
}

# Perform Bayesian Optimization
bayes_search = BayesSearchCV(
    svm, search_space, 
    n_iter=50,  # Number of parameter evaluations
    cv=5, 
    scoring='accuracy', 
    n_jobs=-1, 
    random_state=42
)
bayes_search.fit(X_train_smote, y_train_smote)

# Get the best model
best_model = bayes_search.best_estimator_
print("Best Hyperparameters:", bayes_search.best_params_)

# Evaluate on test set
y_pred = best_model.predict(X_test)
y_proba = best_model.predict_proba(X_test)[:, 1]

print("Test Accuracy:", accuracy_score(y_test, y_pred))
print("ROC AUC Score:", roc_auc_score(y_test, y_proba))
print("Classification Report:\n", classification_report(y_test, y_pred))





i:\Python313\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] The system cannot find the file specified
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "i:\Python313\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
        "wmic CPU Get NumberOfCores /Format:csv".split(),
        capture_output=True,
        text=True,
    )
  File "i:\Python313\Lib\subprocess.py", line 556, in run
    with Popen(*popenargs, **kwargs) as process:
         ~~~~~^^^^^^^^^^^^^^^^^^^^^^
  File "i:\Python313\Lib\subprocess.py", line 1038, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
    ~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
                        

Best Hyperparameters: OrderedDict({'C': 100.0, 'class_weight': 'balanced', 'gamma': 0.0001, 'kernel': 'sigmoid'})
Test Accuracy: 0.6530612244897959
ROC AUC Score: 0.7315789473684211
Classification Report:
               precision    recall  f1-score   support

           0       0.54      0.74      0.62        19
           1       0.78      0.60      0.68        30

    accuracy                           0.65        49
   macro avg       0.66      0.67      0.65        49
weighted avg       0.69      0.65      0.66        49



In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
from sklearn.feature_selection import RFE
from skopt import BayesSearchCV
from skopt.space import Integer, Real
from sklearn.metrics import accuracy_score

# Load dataset (Replace 'your_data.csv' with the actual filename)
df = pd.read_csv('I:/adhd/datasets/combined_featuresECEO.csv')

# Separate features and target
X = df.drop(columns=['ADHD'])  # Assuming 'target' is the label column
y = df['ADHD']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Apply SMOTE to balance the dataset
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Apply Feature Scaling
scaler = StandardScaler()
X_train_smote_scaled = scaler.fit_transform(X_train_smote)
X_test_scaled = scaler.transform(X_test)

# Feature Selection using Recursive Feature Elimination (RFE) with XGBoost
base_xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
rfe = RFE(estimator=base_xgb, n_features_to_select=15)  # Keep top 15 features
X_train_selected = rfe.fit_transform(X_train_smote_scaled, y_train_smote)
X_test_selected = rfe.transform(X_test_scaled)

# Define hyperparameter search space for Bayesian Optimization
param_grid = {
    'n_estimators': Integer(50, 500),  # Number of boosting rounds
    'max_depth': Integer(3, 12),  # Tree depth
    'learning_rate': Real(0.01, 0.3, prior='log-uniform'),  # Step size shrinkage
    'subsample': Real(0.5, 1.0),  # Fraction of samples per tree
    'colsample_bytree': Real(0.5, 1.0),  # Fraction of features per tree
    'gamma': Real(0, 5.0),  # Minimum loss reduction required for a split
    'reg_lambda': Real(1e-3, 10, prior='log-uniform')  # L2 regularization
}

# Initialize XGBoost model
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)

# Perform hyperparameter tuning using Bayesian Optimization
bayes_search = BayesSearchCV(xgb_model, param_grid, n_iter=50, cv=5, scoring='accuracy', n_jobs=-1, random_state=42)
bayes_search.fit(X_train_selected, y_train_smote)

# Best model from the search
best_xgb = bayes_search.best_estimator_

# Evaluate on test set
y_pred = best_xgb.predict(X_test_selected)
test_accuracy = accuracy_score(y_test, y_pred)

print("Best Parameters:", bayes_search.best_params_)
print("Test Accuracy:", test_accuracy)


In [ ]:
import optuna
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from imblearn.over_sampling import SMOTE
from sklearn.feature_selection import RFECV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# Load your dataset
df = pd.read_csv("I:/adhd/datasets/combined_featuresECEO.csv")  # Update with your dataset

# Splitting features and target
X = df.drop(columns=['ADHD'])  # Replace 'target' with your actual column name
y = df['ADHD']

# Apply SMOTE for class imbalance
smote = SMOTE(sampling_strategy='auto', random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Feature Scaling
scaler = StandardScaler()
X_resampled = scaler.fit_transform(X_resampled)

# Feature Selection with RFECV (Recursive Feature Elimination with Cross-Validation)
base_model = XGBClassifier(use_label_encoder=False, eval_metric="logloss", random_state=42)
selector = RFECV(base_model, step=1, cv=3, scoring='accuracy', n_jobs=-1)
X_selected = selector.fit_transform(X_resampled, y_resampled)

# Get the selected feature names
selected_features = X.columns[selector.support_]

# Split Data
X_train, X_test, y_train, y_test = train_test_split(X_selected, y_resampled, test_size=0.2, random_state=42)

# Define Optuna Objective Function
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
        'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_loguniform('gamma', 0.01, 1.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-3, 10.0),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-3, 10.0),
    }
    
    model = XGBClassifier(**params, use_label_encoder=False, eval_metric="logloss", random_state=42)
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy', n_jobs=-1).mean()
    return score

# Run Optuna Optimization
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)  # Change `n_trials` as needed

# Get Best Parameters
best_params = study.best_params
print("Best Hyperparameters:", best_params)

# Train XGBoost Model with Best Parameters
best_model = XGBClassifier(**best_params, use_label_encoder=False, eval_metric="logloss", random_state=42)
best_model.fit(X_train, y_train)



# Evaluate on Test Data
accuracy = best_model.score(X_test, y_test)
print("Final Test Accuracy:", accuracy)


i:\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
i:\Python313\Lib\site-packages\xgboost\training.py:183: UserWarning: [01:02:28] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
i:\Python313\Lib\site-packages\xgboost\training.py:183: UserWarning: [01:02:29] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
i:\Python313\Lib\site-packages\xgboost\training.py:183: UserWarning: [01:02:30] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
i:\Python313\Lib\site-package

Best Hyperparameters: {'n_estimators': 482, 'max_depth': 3, 'learning_rate': 0.13935875596020206, 'subsample': 0.5729662857123876, 'colsample_bytree': 0.9877896868094707, 'gamma': 0.013941617298173203, 'reg_lambda': 2.4184265811835517, 'reg_alpha': 0.009996904274154051}
Final Test Accuracy: 0.8666666666666667


In [ ]:
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from imblearn.over_sampling import SMOTE
from sklearn.feature_selection import RFECV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from joblib import dump
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc


def plot_confusion_matrix(y_true, y_pred, class_names):
    cm = confusion_matrix(y_true, y_pred)
    
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
    
    plt.xlabel("Predicted Labels")
    plt.ylabel("True Labels")
    plt.title("Confusion Matrix")
    plt.show()




def plot_roc_curve(y_true, y_probs):
    fpr, tpr, _ = roc_curve(y_true, y_probs)
    roc_auc = auc(fpr, tpr)

    plt.figure(figsize=(6, 5))
    plt.plot(fpr, tpr, color='blue', label=f'AUC = {roc_auc:.2f}')
    plt.plot([0, 1], [0, 1], color="grey", linestyle="--")  # Diagonal line

    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("ROC Curve")
    plt.legend()
    plt.show()




# Load your dataset
df = pd.read_csv("I:/adhd/datasets/combined_featuresECEO.csv")  # Update with your dataset

# Splitting features and target
X = df.drop(columns=['ADHD'])  # Replace 'target' with your actual column name
y = df['ADHD']

# Apply SMOTE for class imbalance
smote = SMOTE(sampling_strategy='auto', random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Feature Scaling
scaler = StandardScaler()
X_resampled = scaler.fit_transform(X_resampled)

# Feature Selection with RFECV (Recursive Feature Elimination with Cross-Validation)
base_model = XGBClassifier(use_label_encoder=False, eval_metric="logloss", random_state=42)
selector = RFECV(base_model, step=1, cv=3, scoring='accuracy', n_jobs=-1)
X_selected = selector.fit_transform(X_resampled, y_resampled)

# Get the selected feature names
selected_features = X.columns[selector.support_]

# Split Data
X_train, X_test, y_train, y_test = train_test_split(X_selected, y_resampled, test_size=0.2, random_state=42)

best_model = XGBClassifier(n_estimators = 482, max_depth = 3, learning_rate = 0.13935875596020206, subsample = 0.5729662857123876, colsample_bytree = 0.9877896868094707, gamma = 0.013941617298173203, reg_lambda = 2.4184265811835517, reg_alpha = 0.009996904274154051, use_label_encoder=False, eval_metric="logloss", random_state=42)
best_model.fit(X_train, y_train)

y_pred = best_model.predict(X_test)
y_proba = best_model.predict_proba(X_test)[:, 1]

print("Test Accuracy:", accuracy_score(y_test, y_pred))
print("ROC AUC Score:", roc_auc_score(y_test, y_proba))
print("Classification Report:\n", classification_report(y_test, y_pred))

# Evaluate on Test Data
accuracy = best_model.score(X_test, y_test)
print("Final Test Accuracy:", accuracy)





i:\Python313\Lib\site-packages\xgboost\training.py:183: UserWarning: [14:24:43] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
i:\Python313\Lib\site-packages\xgboost\training.py:183: UserWarning: [14:24:44] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
i:\Python313\Lib\site-packages\xgboost\training.py:183: UserWarning: [14:24:44] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
i:\Python313\Lib\site-packages\xgboost\training.py:183: UserWarning: [14:24:45] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
i:\Python313\Lib\sit

Test Accuracy: 0.8666666666666667
ROC AUC Score: 0.9499443826473859
Classification Report:
               precision    recall  f1-score   support

           0       0.85      0.90      0.88        31
           1       0.89      0.83      0.86        29

    accuracy                           0.87        60
   macro avg       0.87      0.87      0.87        60
weighted avg       0.87      0.87      0.87        60

Final Test Accuracy: 0.8666666666666667


['model.joblib']